- Use the credit default dataset and build a decision tree model and a random forest model for predicting "default" outcome based on the others. 
- What is the Gini score at the root?
- Tune the decision tree model to find the combination of max depth and the criterion that maximize the accuracy score.
- As per decision tree model, which are most important features? 
- Tune the random forest model to find the max depth, n_estimators, min_samples_leaf, and criterion that maximize the recall score.


In [2]:
import pandas as pd
from sklearn import * 
import numpy as np

import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
df = pd.read_csv("/data/credit-default.csv")
df.head()

,checking_balance,months_loan_duration,credit_history,purpose,amount,savings_balance,employment_length,installment_rate,personal_status,other_debtors,...,property,age,installment_plan,housing,existing_credits,default,dependents,telephone,foreign_worker,job
0,< 0 DM,6,critical,radio/tv,1169,unknown,> 7 yrs,4,single male,none,...,real estate,67,none,own,2,1,1,yes,yes,skilled employee
1,1 - 200 DM,48,repaid,radio/tv,5951,< 100 DM,1 - 4 yrs,2,female,none,...,real estate,22,none,own,1,2,1,none,yes,skilled employee
2,unknown,12,critical,education,2096,< 100 DM,4 - 7 yrs,2,single male,none,...,real estate,49,none,own,1,1,2,none,yes,unskilled resident
3,< 0 DM,42,repaid,furniture,7882,< 100 DM,4 - 7 yrs,2,single male,guarantor,...,building society savings,45,none,for free,1,1,2,none,yes,skilled employee
4,< 0 DM,24,delayed,car (new),4870,< 100 DM,1 - 4 yrs,3,single male,none,...,unknown/none,53,none,for free,2,2,2,none,yes,skilled employee


In [13]:
%%time 
target = "default"
label_encoder = preprocessing.LabelEncoder()

y = label_encoder.fit_transform(df[target])
X = df.drop(columns=target)
X = pd.get_dummies(X, drop_first=True)

pipe = pipeline.Pipeline([
    #("poly", preprocessing.PolynomialFeatures(degree=2, include_bias=False)),
    #("scaler", preprocessing.StandardScaler()),
    ("est", ensemble.RandomForestClassifier(random_state = 1))
])

param_grid = {
    "est__max_depth": np.arange(7, 15),
    "est__n_estimators": np.arange(15, 30, 2),
    "est__min_samples_leaf": np.arange(2, 10),
    "est__criterion": ["gini", "entropy"],
    "est__max_features": np.linspace(0.7, 0.95, 10)
}

gsearch = model_selection.GridSearchCV(pipe, param_grid, cv = 5, n_jobs=12
                                    , scoring = "accuracy", verbose=True)
gsearch.fit(X, y)

print("best score:", gsearch.best_score_)
print("best param:", gsearch.best_params_)

Fitting 5 folds for each of 10240 candidates, totalling 51200 fits


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    2.2s
[Parallel(n_jobs=12)]: Done 455 tasks      | elapsed:    4.8s
[Parallel(n_jobs=12)]: Done 1455 tasks      | elapsed:   11.2s
[Parallel(n_jobs=12)]: Done 2855 tasks      | elapsed:   21.5s
[Parallel(n_jobs=12)]: Done 4655 tasks      | elapsed:   35.3s
[Parallel(n_jobs=12)]: Done 6855 tasks      | elapsed:   54.0s
[Parallel(n_jobs=12)]: Done 9455 tasks      | elapsed:  1.3min
[Parallel(n_jobs=12)]: Done 12455 tasks      | elapsed:  1.7min
[Parallel(n_jobs=12)]: Done 15855 tasks      | elapsed:  2.2min
[Parallel(n_jobs=12)]: Done 19655 tasks      | elapsed:  2.8min
[Parallel(n_jobs=12)]: Done 23855 tasks      | elapsed:  3.4min
[Parallel(n_jobs=12)]: Done 28455 tasks      | elapsed:  4.1min
[Parallel(n_jobs=12)]: Done 33455 tasks      | elapsed:  4.8min
[Parallel(n_jobs=12)]: Done 38855 tasks      | elapsed:  5.7min
[Parallel(n_jobs=12)]: Done 44655 ta

best score: 0.773
best param: {'est__criterion': 'entropy', 'est__max_depth': 9, 'est__max_features': 0.7277777777777777, 'est__min_samples_leaf': 2, 'est__n_estimators': 25}
CPU times: user 2min 32s, sys: 3.22 s, total: 2min 35s
Wall time: 7min 40s
